In [ ]:
# !pip install openrouteservice geopandas pandas folium matplotlib 

In [338]:
import openrouteservice as ors
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
from folium.plugins import BeautifyIcon

import copy

In [339]:
import constants
import fleet_manager as fm
import utils


In [340]:
pd.options.display.max_rows = 1000    # Show up to 1000 rows
pd.options.display.max_columns = 50   # Show up to 50 columns
pd.set_option('display.max_colwidth', None) # Show all contents in the cells

In [341]:
# change the decimal precision --> default = 6 digits
gpd.options.display_precision = 16 

In [342]:
# create 2-3 postcode options, and prepare pothole datasets accordingly.
"LS9 0BT", "LS6 1PP", "LS6 3HE"
postcode = 'LS9 0BT'

# only options: 1, 2, 3 (drop-down menu)
bot_count = 2

customise = False

In [343]:
client = ors.Client(constants.API_KEY)
client

<table border=1>
<tr>
<td><b>Library	<td><b>Coordinate Order	
<tr>
<td>Folium	<td>Latitude, Longitude	
<tr>
<td>ORS	<td>Longitude, Latitude	

In [344]:
# find the postcode coordinates
station = client.pelias_search(text=postcode)
station

{'geocoding': {'version': '0.2',
  'attribution': 'https://openrouteservice.org/terms-of-service/#attribution-geocode',
  'query': {'text': 'LS9 0BT',
   'size': 10,
   'private': False,
   'lang': {'name': 'English',
    'iso6391': 'en',
    'iso6393': 'eng',
    'via': 'default',
    'defaulted': True},
   'querySize': 20,
   'parser': 'libpostal',
   'parsed_text': {'postalcode': 'ls9 0bt'}},
  'engine': {'name': 'Pelias', 'author': 'Mapzen', 'version': '1.0'},
  'timestamp': 1713028497129},
 'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.517644, 53.789386]},
   'properties': {'id': '470842435',
    'gid': 'whosonfirst:postalcode:470842435',
    'layer': 'postalcode',
    'source': 'whosonfirst',
    'source_id': '470842435',
    'name': 'LS9 0BT',
    'postalcode': 'LS9 0BT',
    'postalcode_gid': 'whosonfirst:postalcode:470842435',
    'confidence': 1,
    'match_type': 'exact',
    'accuracy': 'centroid',
    'co

In [345]:
station_coords = station['features'][0]['geometry']['coordinates']
station_coords

[-1.517644, 53.789386]

In [346]:
m = folium.Map(location=list(reversed(station_coords)), tiles="cartodbpositron", zoom_start=16)

# Create icon using folium.Icon for station from where all bots start and end
icon=BeautifyIcon(
        icon_shape='marker',
        text_color='black',
        background_color="#FFF",
        inner_icon_style="font-size:12px;padding-top:-5px;",
        icon='flag-checkered'
    )

# Add marker with the created icon to the map
folium.Marker(location=list(reversed(station_coords)), icon=icon).add_to(m)

# Display the map
m

In [347]:
fleet = fm.FleetManager(bot_count=bot_count, start=list(reversed(station_coords)))
fleet

In [348]:
potholes_details = gpd.read_file('LS90BT_potholes_small.json')
potholes_details

defect          dim                                         geometry
0  pothole    1x0.5x0.2  POINT (-1.5202653014356144 53.7886094292194912)
1  pothole  0.5x1.2x0.2  POINT (-1.5185276122489597 53.7884573448203867)
2  pothole  0.5x1.2x0.2  POINT (-1.5181200061435902 53.7886537870654990)
3  pothole    1x0.5x0.2  POINT (-1.5165367499993749 53.7894171973833295)
4  pothole    1x0.5x0.2  POINT (-1.5158937220954840 53.7902010524229013)
5  pothole    2x0.5x0.2  POINT (-1.5157207769570675 53.7901240147059525)
6  pothole    1x0.5x0.2  POINT (-1.5150777490531766 53.7897200289646946)
7  pothole    1x0.5x0.2  POINT (-1.5165475271180355 53.7900878370366797)
8  pothole    1x0.5x0.1  POINT (-1.5178539965092739 53.7884356902426077)

In [349]:
potholes_details.describe()

defect        dim                                       geometry
count         9          9                                              9
unique        1          4                                              9
top     pothole  1x0.5x0.2  POINT (-1.5202653014356144 53.78860942921949)
freq          9          5                                              1

In [350]:
potholes_details[['defect']].value_counts()

defect 
pothole    9
dtype: int64

In [351]:
potholes_details[['dim']].value_counts()

dim        
1x0.5x0.2      5
0.5x1.2x0.2    2
1x0.5x0.1      1
2x0.5x0.2      1
dtype: int64

In [352]:
potholes_data = pd.read_json('LS90BT_potholes_small.json')
potholes_data

type  \
0  FeatureCollection   
1  FeatureCollection   
2  FeatureCollection   
3  FeatureCollection   
4  FeatureCollection   
5  FeatureCollection   
6  FeatureCollection   
7  FeatureCollection   
8  FeatureCollection   

                                                                                                                                                                         features  
0    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.520265301435614, 53.78860942921949], 'type': 'Point'}, 'id': 0}  
1  {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '0.5x1.2x0.2'}, 'geometry': {'coordinates': [-1.518527612248959, 53.78845734482039], 'type': 'Point'}, 'id': 1}  
2    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '0.5x1.2x0.2'}, 'geometry': {'coordinates': [-1.51812000614359, 53.7886537870655], 'type': 'Point'}, 'id': 2}  
3    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.516536749999375, 53.78941719738333], 'type': 'Point'}, 'id': 3}  
4     {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.515893722095484, 53.7902010524229], 'type': 'Point'}, 'id': 4}  
5    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '2x0.5x0.2'}, 'geometry': {'coordinates': [-1.515720776957067, 53.79012401470595], 'type': 'Point'}, 'id': 5}  
6   {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.515077749053176, 53.789720028964695], 'type': 'Point'}, 'id': 6}  
7    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.516547527118035, 53.79008783703668], 'type': 'Point'}, 'id': 7}  
8    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.1'}, 'geometry': {'coordinates': [-1.517853996509273, 53.78843569024261], 'type': 'Point'}, 'id': 8}

In [353]:
ut = utils.Utils()
ut

In [357]:
# Function to get job details for a pothole feature
def create_job(feature):
    volume, surface_area, qty_reqd, setup_time, service_time = ut.calculate_eta(feature)
    return ors.optimization.Job(location=feature['geometry']['coordinates'],
                                id=feature['id'],
                                setup=setup_time,
                                service=service_time,
                                amount=[qty_reqd])

In [358]:
# Function to get pothole estimates
def get_pothole_details(feature):
    volume, surface_area, qty_reqd, setup_time, service_time = ut.calculate_eta(feature)
    return {'id': feature['id'],
            'defect': feature['properties']['defect'],
            'volume': volume,
            'surface_area': surface_area,
            'qty_reqd': qty_reqd,
            'setup_time': setup_time,
            'service_time': service_time}

In [356]:
# Iterate through pothole data to calculate estimates and store job details

jobs = []
pothole_estimates = []

for feature in potholes_data['features']:
    jobs.append(create_job(feature)) 
    pothole_estimates.append(get_pothole_details(feature))

In [290]:
pothole_estimates

[{'id': 0,
  'defect': 'pothole',
  'volume': 0.1,
  'surface_area': 0.5,
  'qty_reqd': 240.0,
  'setup_time': 600,
  'service_time': 270.0},
 {'id': 1,
  'defect': 'pothole',
  'volume': 0.12,
  'surface_area': 0.6,
  'qty_reqd': 288.0,
  'setup_time': 600,
  'service_time': 324.0},
 {'id': 2,
  'defect': 'pothole',
  'volume': 0.12,
  'surface_area': 0.6,
  'qty_reqd': 288.0,
  'setup_time': 600,
  'service_time': 324.0},
 {'id': 3,
  'defect': 'pothole',
  'volume': 0.1,
  'surface_area': 0.5,
  'qty_reqd': 240.0,
  'setup_time': 600,
  'service_time': 270.0},
 {'id': 4,
  'defect': 'pothole',
  'volume': 0.1,
  'surface_area': 0.5,
  'qty_reqd': 240.0,
  'setup_time': 600,
  'service_time': 270.0},
 {'id': 5,
  'defect': 'pothole',
  'volume': 0.2,
  'surface_area': 1.0,
  'qty_reqd': 480.0,
  'setup_time': 600,
  'service_time': 540.0},
 {'id': 6,
  'defect': 'pothole',
  'volume': 0.1,
  'surface_area': 0.5,
  'qty_reqd': 240.0,
  'setup_time': 600,
  'service_time': 270.0},
 {'i

### Potholes estimates

In [291]:
pedf = pd.DataFrame(pothole_estimates)
pedf

id   defect  volume  surface_area  qty_reqd  setup_time  service_time
0   0  pothole    0.10           0.5     240.0         600         270.0
1   1  pothole    0.12           0.6     288.0         600         324.0
2   2  pothole    0.12           0.6     288.0         600         324.0
3   3  pothole    0.10           0.5     240.0         600         270.0
4   4  pothole    0.10           0.5     240.0         600         270.0
5   5  pothole    0.20           1.0     480.0         600         540.0
6   6  pothole    0.10           0.5     240.0         600         270.0
7   7  pothole    0.10           0.5     240.0         600         270.0
8   8  pothole    0.05           0.5     120.0         600         225.0

In [328]:
# plot potholes(jobs) and routes

for pothole in potholes_data['features']:
    folium.Marker(
        location=list(reversed(pothole['geometry']['coordinates'])),
        tooltip=folium.map.Tooltip(
            """<table>
            <tr><td>ID: <td>{}
            <tr><td>Defect: <td>{}
            <tr><td>Volume: <b><td>{} m<sup>3</sup>
            <tr><td>Surface Area: <b><td>{} m<sup>2</sup>
            <tr><td>Material Required: <b><td>{} kg
            """.format(
                pothole['id'],
                pothole['properties']['defect'],
                pedf[pedf['id']==pothole['id']]['volume'].values[0],
                pedf[pedf['id']==pothole['id']]['surface_area'].values[0],
                pedf[pedf['id']==pothole['id']]['qty_reqd'].values[0]
                )
            , opacity=1),
        icon=BeautifyIcon(
            # icon_shape='marker',
            # number=pothole['id'],
            text_color='white',
            background_color="red",
            inner_icon_style="font-size:12px;padding-top:-5px;",
            icon='info'
        )
        # add details in tooltip
        # add routes
        # add layer control
    ).add_to(m)

m

In [330]:
jobs

In [331]:
# plan route using ORS
route_plan = ors.optimization.optimization(client,
                                  jobs=jobs,
                                  vehicles=fleet.bots,
                                  geometry=True)

# if you get error code 500 from ORS API, check location coordinate format.

route_plan

{'code': 0,
 'summary': {'cost': 357,
  'routes': 2,
  'unassigned': 1,
  'delivery': [1896],
  'amount': [1896],
  'pickup': [0],
  'setup': 0,
  'service': 2223,
  'duration': 357,
  'waiting_time': 0,
  'priority': 0,
  'distance': 1945,
  'violations': [],
  'computing_times': {'loading': 106, 'solving': 78, 'routing': 53}},
 'unassigned': [{'id': 5,
   'location': [-1.515720776957067, 53.79012401470595],
   'type': 'job'}],
 'routes': [{'vehicle': 0,
   'cost': 174,
   'delivery': [960],
   'amount': [960],
   'pickup': [0],
   'setup': 0,
   'service': 1080,
   'duration': 174,
   'waiting_time': 0,
   'priority': 0,
   'distance': 887,
   'steps': [{'type': 'start',
     'location': [-1.517644, 53.789386],
     'setup': 0,
     'service': 0,
     'waiting_time': 0,
     'load': [960],
     'arrival': 0,
     'duration': 0,
     'violations': [],
     'distance': 0},
    {'type': 'job',
     'location': [-1.516536749999375, 53.78941719738333],
     'id': 3,
     'setup': 0,
     

In [332]:
unassigned_jobs = [job['id'] for job in route_plan['unassigned']] # ids of unassigned jobs
remaining_jobs = [job for job in jobs if job.id in unassigned_jobs]

remaining_jobs

In [333]:
# plan route using ORS
route_plan_2 = ors.optimization.optimization(client,
                                  jobs=remaining_jobs,
                                  vehicles=fleet.bots,
                                  shipments=None,
                                  matrix=None,
                                  geometry=True,
                                  dry_run=None)
# if you get error code 500 from ORS API, check location coordinate format.
# error code 4, if bot_count > 3

route_plan_2

{'code': 0,
 'summary': {'cost': 126,
  'routes': 1,
  'unassigned': 0,
  'delivery': [480],
  'amount': [480],
  'pickup': [0],
  'setup': 0,
  'service': 540,
  'duration': 126,
  'waiting_time': 0,
  'priority': 0,
  'distance': 686,
  'violations': [],
  'computing_times': {'loading': 61, 'solving': 49, 'routing': 14}},
 'unassigned': [],
 'routes': [{'vehicle': 0,
   'cost': 126,
   'delivery': [480],
   'amount': [480],
   'pickup': [0],
   'setup': 0,
   'service': 540,
   'duration': 126,
   'waiting_time': 0,
   'priority': 0,
   'distance': 686,
   'steps': [{'type': 'start',
     'location': [-1.517644, 53.789386],
     'setup': 0,
     'service': 0,
     'waiting_time': 0,
     'load': [480],
     'arrival': 0,
     'duration': 0,
     'violations': [],
     'distance': 0},
    {'type': 'job',
     'location': [-1.515720776957067, 53.79012401470595],
     'id': 5,
     'setup': 0,
     'service': 540,
     'waiting_time': 0,
     'job': 5,
     'load': [0],
     'arrival': 

In [334]:
plan_count = 1

In [335]:
# TODO
# check for how many jobs are left undone
# try with higher number of bots
# try with higher capacity
#  try with bigger time_window

In [336]:
line_colours = ['orange', 'blue', 'green', 'pink', 'yellow']

for route in route_plan['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], weight=2, color=line_colours[route['vehicle']], opacity=0.5).add_to(m)

m

In [337]:
line_colours = ['orange', 'blue', 'green', 'pink', 'yellow']

for route in route_plan['routes']:
    r1 = folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], weight=5, color=line_colours[route['vehicle']+bot_count*plan_count], opacity=0.5)

m

In [228]:
route_plan

{'code': 0,
 'summary': {'cost': 452,
  'routes': 3,
  'unassigned': 0,
  'delivery': [2376],
  'amount': [2376],
  'pickup': [0],
  'setup': 0,
  'service': 2763,
  'duration': 452,
  'waiting_time': 0,
  'priority': 0,
  'distance': 2432,
  'violations': [],
  'computing_times': {'loading': 88, 'solving': 41, 'routing': 43}},
 'unassigned': [],
 'routes': [{'vehicle': 0,
   'cost': 133,
   'delivery': [960],
   'amount': [960],
   'pickup': [0],
   'setup': 0,
   'service': 1080,
   'duration': 133,
   'waiting_time': 0,
   'priority': 0,
   'distance': 716,
   'steps': [{'type': 'start',
     'location': [-1.517644, 53.789386],
     'setup': 0,
     'service': 0,
     'waiting_time': 0,
     'load': [960],
     'arrival': 0,
     'duration': 0,
     'violations': [],
     'distance': 0},
    {'type': 'job',
     'location': [-1.515077749053176, 53.789720028964695],
     'id': 6,
     'setup': 0,
     'service': 270,
     'waiting_time': 0,
     'job': 6,
     'load': [720],
     'ar

In [229]:
pd.DataFrame([route['steps'] for route in route_plan['routes']])

0  \
0  {'type': 'start', 'location': [-1.517644, 53.789386], 'setup': 0, 'service': 0, 'waiting_time': 0, 'load': [960], 'arrival': 0, 'duration': 0, 'violations': [], 'distance': 0}   
1  {'type': 'start', 'location': [-1.517644, 53.789386], 'setup': 0, 'service': 0, 'waiting_time': 0, 'load': [720], 'arrival': 0, 'duration': 0, 'violations': [], 'distance': 0}   
2  {'type': 'start', 'location': [-1.517644, 53.789386], 'setup': 0, 'service': 0, 'waiting_time': 0, 'load': [696], 'arrival': 0, 'duration': 0, 'violations': [], 'distance': 0}   

                                                                                                                                                                                                                          1  \
0  {'type': 'job', 'location': [-1.515077749053176, 53.789720028964695], 'id': 6, 'setup': 0, 'service': 270, 'waiting_time': 0, 'job': 6, 'load': [720], 'arrival': 48, 'duration': 48, 'violations': [], 'distance': 281}   
1   {'type': 'job', 'location': [-1.520265301435614, 53.78860942921949], 'id': 0, 'setup': 0, 'service': 270, 'waiting_time': 0, 'job': 0, 'load': [480], 'arrival': 74, 'duration': 74, 'violations': [], 'distance': 430}   
2     {'type': 'job', 'location': [-1.51812000614359, 53.7886537870655], 'id': 2, 'setup': 0, 'service': 324, 'waiting_time': 0, 'job': 2, 'load': [408], 'arrival': 18, 'duration': 18, 'violations': [], 'distance': 108}   

                                                                                                                                                                                                                           2  \
0    {'type': 'job', 'location': [-1.515893722095484, 53.7902010524229], 'id': 4, 'setup': 0, 'service': 270, 'waiting_time': 0, 'job': 4, 'load': [480], 'arrival': 336, 'duration': 66, 'violations': [], 'distance': 358}   
1  {'type': 'job', 'location': [-1.516547527118035, 53.79008783703668], 'id': 7, 'setup': 0, 'service': 270, 'waiting_time': 0, 'job': 7, 'load': [240], 'arrival': 438, 'duration': 168, 'violations': [], 'distance': 989}   
2   {'type': 'job', 'location': [-1.518527612248959, 53.78845734482039], 'id': 1, 'setup': 0, 'service': 324, 'waiting_time': 0, 'job': 1, 'load': [120], 'arrival': 346, 'duration': 22, 'violations': [], 'distance': 143}   

                                                                                                                                                                                                                          3  \
0    {'type': 'job', 'location': [-1.515720776957067, 53.79012401470595], 'id': 5, 'setup': 0, 'service': 540, 'waiting_time': 0, 'job': 5, 'load': [0], 'arrival': 610, 'duration': 70, 'violations': [], 'distance': 373}   
1  {'type': 'job', 'location': [-1.516536749999375, 53.78941719738333], 'id': 3, 'setup': 0, 'service': 270, 'waiting_time': 0, 'job': 3, 'load': [0], 'arrival': 733, 'duration': 193, 'violations': [], 'distance': 1094}   
2    {'type': 'job', 'location': [-1.517853996509273, 53.78843569024261], 'id': 8, 'setup': 0, 'service': 225, 'waiting_time': 0, 'job': 8, 'load': [0], 'arrival': 684, 'duration': 36, 'violations': [], 'distance': 202}   

                                                                                                                                                                                     4  
0   {'type': 'end', 'location': [-1.517644, 53.789386], 'setup': 0, 'service': 0, 'waiting_time': 0, 'load': [0], 'arrival': 1213, 'duration': 133, 'violations': [], 'distance': 716}  
1  {'type': 'end', 'location': [-1.517644, 53.789386], 'setup': 0, 'service': 0, 'waiting_time': 0, 'load': [0], 'arrival': 1059, 'duration': 249, 'violations': [], 'distance': 1374}  
2     {'type': 'end', 'location': [-1.517644, 53.789386], 'setup': 0, 'service': 0, 'waiting_time': 0, 'load': [0], 'arrival': 943, 'duration': 70, 'violations': [], 'distance': 342}

In [231]:
# Define an empty list to store route data
route_data = []

# Loop through each route in the 'routes' list
for route in route_plan['routes']:
    vehicle_num = route['vehicle']  # Get vehicle number

    # Loop through each step (job) within the route
    for step in route['steps']:
        # Extract relevant data from each step
        step_data = {
            'vehicle': vehicle_num,
            'type': step['type'],
            'location': step['location'],
            'setup': step['setup'],
            'service': step['service'],
            'waiting_time': step['waiting_time'],
            'load': step['load'],
            'arrival': step['arrival'],
            'duration': step['duration'],
            'violations': step['violations'],
            'distance': step['distance'],
            'id': step.get('id', None),
            'job': step.get('job', None)
        }

        # Append the data for this step to the route_data list
        route_data.append(step_data)

# Create DataFrame from the route_data list
df = pd.DataFrame(route_data)

# Print the DataFrame
df

vehicle   type                                  location  setup  service  \
0         0  start                    [-1.517644, 53.789386]      0        0   
1         0    job  [-1.515077749053176, 53.789720028964695]      0      270   
2         0    job    [-1.515893722095484, 53.7902010524229]      0      270   
3         0    job   [-1.515720776957067, 53.79012401470595]      0      540   
4         0    end                    [-1.517644, 53.789386]      0        0   
5         1  start                    [-1.517644, 53.789386]      0        0   
6         1    job   [-1.520265301435614, 53.78860942921949]      0      270   
7         1    job   [-1.516547527118035, 53.79008783703668]      0      270   
8         1    job   [-1.516536749999375, 53.78941719738333]      0      270   
9         1    end                    [-1.517644, 53.789386]      0        0   
10        2  start                    [-1.517644, 53.789386]      0        0   
11        2    job     [-1.51812000614359, 53.7886537870655]      0      324   
12        2    job   [-1.518527612248959, 53.78845734482039]      0      324   
13        2    job   [-1.517853996509273, 53.78843569024261]      0      225   
14        2    end                    [-1.517644, 53.789386]      0        0   

    waiting_time   load  arrival  duration violations  distance   id  job  
0              0  [960]        0         0         []         0  NaN  NaN  
1              0  [720]       48        48         []       281  6.0  6.0  
2              0  [480]      336        66         []       358  4.0  4.0  
3              0    [0]      610        70         []       373  5.0  5.0  
4              0    [0]     1213       133         []       716  NaN  NaN  
5              0  [720]        0         0         []         0  NaN  NaN  
6              0  [480]       74        74         []       430  0.0  0.0  
7              0  [240]      438       168         []       989  7.0  7.0  
8              0    [0]      733       193         []      1094  3.0  3.0  
9              0    [0]     1059       249         []      1374  NaN  NaN  
10             0  [696]        0         0         []         0  NaN  NaN  
11             0  [408]       18        18         []       108  2.0  2.0  
12             0  [120]      346        22         []       143  1.0  1.0  
13             0    [0]      684        36         []       202  8.0  8.0  
14             0    [0]      943        70         []       342  NaN  NaN

In [239]:
# route_info = pd.DataFrame(route_plan)

ValueError: All arrays must be of the same length

In [232]:
# Only extract relevant fields from the response
extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in route_plan['routes']]

bots_df = pd.DataFrame(data)
bots_df = bots_df.rename(columns={'distance': 'distance (km)', 
                                  'amount': 'amount (kg)', 
                                  'duration': 'travel duration (sec)'})
bots_df.index.name = 'vehicle'
bots_df

distance (km) amount (kg)  travel duration (sec)
vehicle                                                  
0                  716       [960]                    133
1                 1374       [720]                    249
2                  342       [696]                     70

In [233]:
# Filter for rows that represent jobs
job_data = df[df['type'] == 'job']

# Group by vehicle and calculate total distance and number of jobs
grouped_data = job_data.groupby('vehicle').agg(total_distance=('distance', 'sum'), num_jobs=('job', 'count'))

# Create charts
fig, ax = plt.subplots(3, 1, figsize=(10, 15))

# Total distance per vehicle
grouped_data['total_distance'].plot(kind='bar', ax=ax[0], color='skyblue', title='Total Distance per Vehicle')
ax[0].set_ylabel('Total Distance')

# Number of jobs per vehicle
grouped_data['num_jobs'].plot(kind='bar', ax=ax[1], color='lightgreen', title='Number of Jobs per Vehicle')
ax[1].set_ylabel('Number of Jobs')

# Distance against time for each job
ax[2].scatter(job_data['duration'], job_data['distance'], c=job_data['vehicle'].map({0: 'red', 1: 'blue', 2: 'green'}))
ax[2].set_title('Distance vs Duration for Jobs')
ax[2].set_xlabel('Duration (min)')
ax[2].set_ylabel


<bound method _AxesBase.set_ylabel of <Axes: title={'center': 'Distance vs Duration for Jobs'}, xlabel='Duration (min)'>>

In [ ]:
# maps, charts, comparisons


In [99]:
# read from the json response

import matplotlib.pyplot as plt

# Data from ORS response
routes = route_plan['routes']

# Lists to store data for charts
distances = []
durations = []
setup_times = []
service_times = []

# Extract data from each route
for route in routes:
  distances.append(route['distance'])
  durations.append(route['duration'])
  for step in route['steps']:
    setup_times.append(step['setup'])
    service_times.append(step['service'])

# Distance vs. Time Chart
plt.figure(figsize=(10, 6))
plt.scatter(distances, durations)
plt.xlabel('Distance (meters)')
plt.ylabel('Duration (seconds)')
plt.title('Distance vs. Time per Route')
plt.grid(True)
plt.savefig('distance_vs_time.png')
plt.close()

# Setup Time vs. Service Time (per bot)
bot_setup_times = {}
bot_service_times = {}

# Separate data by bot (vehicle)
for route in routes:
  vehicle = route['vehicle']
  for step in route['steps']:
    if vehicle not in bot_setup_times:
      bot_setup_times[vehicle] = []
      bot_service_times[vehicle] = []
    bot_setup_times[vehicle].append(step['setup'])
    bot_service_times[vehicle].append(step['service'])

# Create separate charts for each bot
for bot, setup_data in bot_setup_times.items():
  plt.figure(figsize=(8, 5))
  plt.plot(setup_data, label='Setup Time')
  plt.plot(bot_service_times[bot], label='Service Time')
  plt.xlabel('Job Index')
  plt.ylabel('Time (seconds)')
  plt.title(f'Setup vs. Service Time (Bot {bot})')
  plt.legend()
  plt.grid(True)
  plt.savefig(f'bot_{bot}_setup_vs_service.png')
  plt.close()

# Setup Time vs. Service Time (per pothole) - requires pothole IDs
# ... (implement logic to associate setup/service times with pothole IDs)


In [ ]:
# show charts here

In [100]:
# tableau dashboard? --> NO
# instead grid view with 4 or 6 charts.

In [ ]:
# collapse bot_id=1 at time t=2*3600 (2hr)

# re-scheduling for available bots (bot_ids=[0, 2]; [capacity=] [jobs_pending=] [current_location=])

In [ ]:
while True:
    # Perform route optimization
    route_plan = ors.optimization.optimization(client, jobs=jobs, vehicles=fleet.bots, shipments=None, matrix=None, geometry=True, dry_run=None)
    
    # Get unassigned job IDs
    unassigned_job_ids = [job['id'] for job in route_plan['unassigned']]
    
    # Check if there are unassigned jobs
    if unassigned_job_ids:
        print('Unassigned jobs:', unassigned_job_ids)
        
        # Plan for unassigned jobs
        for job_id in unassigned_job_ids:
            # Find an available bot
            available_bot = fleet.get_available_bot()
            if available_bot is not None:
                # Assign the job to the available bot
                job_index = next(index for index, job in enumerate(jobs) if job.id == job_id)
                jobs[job_index].vehicle_id = available_bot.id
    
    else:
        # No unassigned jobs, exit the loop
        break

# Print the final route plan
print("Final route plan:", route_plan)


In [303]:
from enum import Enum

In [360]:
ors_response = route_plan

# Define map center and zoom level
latitude = 53.78  # Replace with your desired center latitude
longitude = -1.52  # Replace with your desired center longitude
zoom_level = 14

# Create a base folium map
m = folium.Map(location=[latitude, longitude], zoom_start=zoom_level)

# Create a dictionary to store decoded route geometries
route_geometries = {}

# Function to decode the geometry string from ORS
def decode_geometry(geometry_string):
  # Implement your logic to decode the geometry string based on ORS format
  # This might involve using a library or custom function
  # Replace the following with your actual decoding logic
  decoded_points = []
  # ... decode geometry string into a list of latitude/longitude coordinates
  return decoded_points

# Process routes and create FeatureGroups
vehicle_layers = []
for route in ors_response['routes']:
  # Decode the route geometry
  decoded_points = decode_geometry(route['geometry'])
  route_geometries[route['vehicle']] = decoded_points  # Store for later use

  # Create a FeatureGroup for the current route
  vehicle_layer = folium.FeatureGroup(name=f"Vehicle {route['vehicle']+1}")
  folium.PolyLine(locations=decoded_points).add_to(vehicle_layer)  # Add route line

  # Add other route information (optional)
  # You can add markers for start/end locations or other relevant data points

  vehicle_layers.append(vehicle_layer)

# Add all FeatureGroups to the map
for layer in vehicle_layers:
  layer.add_to(m)

# Create a LayerControl and add it to the map
folium.LayerControl().add_to(m)

# Display the map
m

ValueError: Locations is empty.